In [1]:
%pip install h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from collections import defaultdict
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.image as mpimg
from torch.utils.data import DataLoader

In [4]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #self.linear_layer_size = linear_layer_size

        self.conv1 = torch.nn.Conv2d(3, 16, kernel_size=3)  # input: torch.Size([64, 3, 96, 96])
        self.conv2 = torch.nn.Conv2d(16, 32, kernel_size=3) # input: torch.Size([64, 16, 47, 47])
        self.conv3 = torch.nn.Conv2d(32, 64, kernel_size=3) # input: torch.Size([64, 32, 22, 22])
        self.conv4 = torch.nn.Conv2d(64, 128, kernel_size=3) # input: torch.Size([64, 64, 10, 10])
        # output: torch.Size([64, 128, 4, 4])
        

    def forward(self, x):
        #print(x.shape)
        batch_size = x.size(0)

        relu = torch.nn.ReLU()
        maxpool = torch.nn.MaxPool2d(stride=2,kernel_size=2)

        x = self.conv1(x)
        x = maxpool(relu(x))
        #print(x.shape)
        x = self.conv2(x)
        x = maxpool(relu(x))
        #print(x.shape)
        x = self.conv3(x)
        x = maxpool(relu(x))
        #print(x.shape)
        x = self.conv4(x)
        x = maxpool(relu(x))
        #print(x.shape)
        # x = x.reshape(batch_size, self.linear_layer_size)
        # x = self.linear(x)
        return x

In [5]:
class FC(torch.nn.Module):
    def __init__(self):
          super(FC, self).__init__()
          self.fc1 = torch.nn.Linear(4 * 4 * 128, 1)  # input: (6,6,128) # output 2 classes
          self.actv = torch.nn.Sigmoid()
    def forward(self, x):
        #print(x.shape)
        batch_size = x.size(0)
        x = torch.flatten(x, 1)
        #print(x.shape)
        x = self.fc1(x)
        x = self.actv(x)
        return x


In [6]:
class RNN(torch.nn.Module):
    def __init__(self, ):
        super(RNN, self).__init__()
        self.input_size = 128
        self.output_size = 2
        self.hidden_size = 32
        self.n_layers = 1
        # batch_first means that the first dim of the input and output will be the batch_size
        self.rnn = torch.nn.RNN(self.input_size, self.hidden_size, self.n_layers, batch_first=True)
        # last, fully-connected layer
        self.fc = torch.nn.Linear(self.hidden_dim, self.output_size)

    def forward(self, x, hidden):
        # x (batch_size, seq_length, input_size)
        # hidden (n_layers, batch_size, hidden_dim)
        # r_out (batch_size, time_step, hidden_size)
        batch_size = x.size(0)
        
        # get RNN outputs
        r_out, hidden = self.rnn(x, hidden)
        # shape output to be (batch_size*seq_length, hidden_dim)
        r_out = r_out.view(-1, self.hidden_size)
        
        # get final output 
        output = self.fc(r_out)
        
        return output, hidden

In [7]:
class LSTM(torch.nn.Module):
    def __init__(self):
        super(LSTM,self).__init__()
        self.input_size = 128
        self.output_size = 2
        self.hidden_size = 32
        self.n_layers = 1
        self.lstm = torch.nn.LSTM(input_size=self.input_size,hidden_size=self.hidden_size,num_layers=self.n_layers,batch_first=True)
        self.fc1 = torch.nn.Linear(in_features=5,out_features=self.output_size)

    def forward(self,x):
        output,_status = self.lstm(x)
        output = output.view(-1, self.hidden_size)
        output = self.fc1(torch.relu(output))
        return output


In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [9]:
train_data = datasets.PCAM(
    root="/content/drive/MyDrive/CS449",
    split="train",
    download=False,
    transform=ToTensor()
)

In [10]:
train_data_small = []
for i in range(100):
  train_data_small.append(train_data[i])

In [11]:
train_dataloader = DataLoader(train_data_small, batch_size=32, shuffle=True)

In [16]:
def run_one_epoch(epoch_idx, model1, model2, optimizer, data_loader, train=True):

    if train:
        model1.train()
        model2.train()
    else:
        model1.eval()
        model2.eval()
    # loop through the entire dataset once per epoch
    train_loss = 0.0
    train_acc = 0.0
    train_total = 0
    for batch_idx, (images, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        model1_output = model1(images)
        #print("hi")
        predictions = model2(model1_output).squeeze()
        acc_sum = 0
        for i, pred in enumerate(predictions):
          if round(pred) == labels[i]:
            acc_sum += 1
        acc = acc_sum / label.size(0)
        # acc = torch.sum(predictions == labels) / labels.size(0)
        loss = torch.nn.BCELoss()(predictions, labels.float())

        if train:
            loss.backward()
            optimizer.step()

    # update statistics
        train_loss += (loss * len(images))
        train_acc += (acc * len(labels))
        train_total += len(labels)

    train_loss /= train_total
    train_acc /= train_total

    print(f"Epoch {epoch_idx + 1}: train loss {train_loss :0.3f}, train acc {train_acc :0.3f}")

    # Detach tells torch to stop tracking a tensor's gradients
    return acc.detach(), loss.detach()

In [13]:
cnn_fc = CNN()
cnn_fc.to(device)
fc = FC()
fc.to(device)

FC(
  (fc1): Linear(in_features=2048, out_features=1, bias=True)
  (actv): Sigmoid()
)

In [14]:
epochs = 100
lr = 1e-6


opt = torch.optim.Adam([{"params": cnn_fc.parameters()},{"params":fc.parameters()}], lr=lr)
for epoch in range(epochs):
  acc, loss = run_one_epoch(epoch+1, cnn_fc, fc, opt, train_dataloader)

Epoch 2: train loss 0.690, train acc 0.000
Epoch 3: train loss 0.690, train acc 0.020
Epoch 4: train loss 0.690, train acc 0.000
Epoch 5: train loss 0.690, train acc 0.000
Epoch 6: train loss 0.690, train acc 0.000
Epoch 7: train loss 0.690, train acc 0.010
Epoch 8: train loss 0.690, train acc 0.000
Epoch 9: train loss 0.690, train acc 0.010
Epoch 10: train loss 0.690, train acc 0.000
Epoch 11: train loss 0.690, train acc 0.210
Epoch 12: train loss 0.690, train acc 0.030
Epoch 13: train loss 0.690, train acc 0.000
Epoch 14: train loss 0.690, train acc 0.120
Epoch 15: train loss 0.689, train acc 0.210
Epoch 16: train loss 0.689, train acc 0.010
Epoch 17: train loss 0.689, train acc 0.200
Epoch 18: train loss 0.689, train acc 0.290
Epoch 19: train loss 0.689, train acc 0.000
Epoch 20: train loss 0.689, train acc 0.030
Epoch 21: train loss 0.689, train acc 0.010
Epoch 22: train loss 0.689, train acc 0.010
Epoch 23: train loss 0.689, train acc 0.030
Epoch 24: train loss 0.689, train acc 0.